<a href="https://colab.research.google.com/github/mishra-yogendra/Pytorch/blob/main/Pytorch_RNN_Based_Q%26A_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [46]:
df.head(1)

,question,answer
0,What is the capital of France?,Paris


In [49]:
#from google.colab import sheets
#sheet = sheets.InteractiveSheet(df=df)

In [2]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [3]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [4]:
# vocab
vocab = {'<UNK>':0}

In [5]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [6]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [7]:
len(vocab)

324

In [8]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [9]:
text_to_indices("What is Pytorch", vocab)

[1, 2, 0]

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

In [11]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [12]:
dataset = QADataset(df, vocab)

In [13]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [14]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 78,  79, 261, 151,  14, 262, 153]]) tensor([36])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([28])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([49])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([113])
tensor([[10, 96,  3, 97]]) tensor([98])
tensor([[  1,   2,   3, 234,   5, 235]]) tensor([131])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([166])
tensor([[  1,   2,   3,  37, 133,   5,  26]]) tensor([134])
tensor([[ 78,  79, 195,  81,  19,   3, 196, 197, 198]]) tensor([199])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[  1,   2,   3,  92, 137,  19,   3,  45]]) tensor([185])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([36])
tensor([[ 1,  2,  3,  4,  5, 73]]) tensor([74])
tensor([[  1,   2,   3,  69,   5, 155]]) tensor([156])
tensor([[ 1,  2,  3, 37, 38, 39, 40]]) tensor([41])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([68])
tensor([[  1,   2,   3,   4,   5, 236, 237]]) tensor([238])
tensor([[  1

In [15]:
import torch.nn as nn

In [17]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [18]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [29]:
learning_rate = 0.001
epochs = 100
model = SimpleRNN(len(vocab))

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [31]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 527.801131
Epoch: 2, Loss: 460.647312
Epoch: 3, Loss: 380.116750
Epoch: 4, Loss: 315.812543
Epoch: 5, Loss: 263.803538
Epoch: 6, Loss: 215.883011
Epoch: 7, Loss: 172.373878
Epoch: 8, Loss: 134.384492
Epoch: 9, Loss: 103.212792
Epoch: 10, Loss: 79.163401
Epoch: 11, Loss: 60.456985
Epoch: 12, Loss: 47.036437
Epoch: 13, Loss: 37.060540
Epoch: 14, Loss: 30.090851
Epoch: 15, Loss: 24.614906
Epoch: 16, Loss: 20.232748
Epoch: 17, Loss: 17.057063
Epoch: 18, Loss: 14.573908
Epoch: 19, Loss: 12.447248
Epoch: 20, Loss: 10.796629
Epoch: 21, Loss: 9.481768
Epoch: 22, Loss: 8.392864
Epoch: 23, Loss: 7.456089
Epoch: 24, Loss: 6.667379
Epoch: 25, Loss: 6.011382
Epoch: 26, Loss: 5.439340
Epoch: 27, Loss: 4.944184
Epoch: 28, Loss: 4.505157
Epoch: 29, Loss: 4.109902
Epoch: 30, Loss: 3.777883
Epoch: 31, Loss: 3.470160
Epoch: 32, Loss: 3.202674
Epoch: 33, Loss: 2.958464
Epoch: 34, Loss: 2.739014
Epoch: 35, Loss: 2.553125
Epoch: 36, Loss: 2.365603
Epoch: 37, Loss: 2.204350
Epoch: 38, Loss: 2

In [32]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [33]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [34]:
predict(model, "What is the boiling point of water in Celsius?")

100


In [35]:
predict(model, "What is the capital of Spain?")

madrid


In [50]:
predict(model, "Who discovered the Americas in 1492?")

christophercolumbus
